In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense

# Step 1: Simulate electricity consumption data
np.random.seed(42)
normal_data = np.random.normal(loc=300, scale=15, size=480)  # normal hours
anomalies = np.array([100, 120, 500, 550, 600])  # fake spikes
consumption = np.concatenate([normal_data, anomalies])
df = pd.DataFrame({'consumption': consumption})

# Normalize data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df[['consumption']])

# Step 2: Build autoencoder model
X_train, X_test = train_test_split(data_scaled, test_size=0.2, random_state=42)

model = Sequential([
    Dense(8, activation='relu', input_shape=(1,)),
    Dense(4, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, X_train, epochs=50, batch_size=16, verbose=0)

# Step 3: Reconstruction and anomaly detection
X_pred = model.predict(data_scaled)
reconstruction_error = np.mean(np.square(X_pred - data_scaled), axis=1)
df['reconstruction_error'] = reconstruction_error
threshold = np.percentile(reconstruction_error, 95)  # Top 5% as anomalies
df['anomaly'] = df['reconstruction_error'] > threshold

# Step 4: Visualize
plt.figure(figsize=(12, 6))
plt.plot(df['consumption'], label='Actual')
plt.plot(scaler.inverse_transform(X_pred), label='Reconstructed', alpha=0.7)
plt.scatter(df[df['anomaly']].index, df[df['anomaly']]['consumption'], color='red', label='Anomaly')
plt.title('Electricity Consumption - Autoencoder Anomaly Detection')
plt.xlabel('Time')
plt.ylabel('Consumption')
plt.legend()
plt.grid(True)
plt.show()

print(f"Total anomalies detected: {df['anomaly'].sum()}")